## Unet
<img src="https://i.imgur.com/LQORH9i.png" alt="drawing" width="400"/>

In [4]:
!pip install -q --user albumentations

In [6]:
import cv2
import numpy as np
import torch # 1.9
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
import matplotlib.pyplot as plt
import albumentations as A
import os
import torch.optim as optim
from utils import *
from unet import UNet

In [9]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
BATCH_SIZE = 32
NUM_LABELS = 1
WIDTH = 512
HEIGHT = 512

In [8]:
unet = UNet(num_class=1, padding = 'same', out_sz=(WIDTH,HEIGHT), retain_dim=False)
unet#.to(device)
x    = torch.randn(1, 3, WIDTH, HEIGHT)#.to(device)
y_pred = unet(x)
y_pred.shape

torch.Size([1, 1, 512, 512])

### 下載資料集 (MVtech)
THE MVTEC ANOMALY DETECTION DATASET (MVTEC AD)


<img src="https://www.mvtec.com/fileadmin/Redaktion/mvtec.com/company/research/datasets/dataset_overview_large.png" alt="drawing" width="400"/>

In [10]:
import os   
cmd = '''mkdir ./data
wget -q https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937454-1629951595/capsule.tar.xz -O data/MVtech-capsule.tar.xz
tar -Jxf data/MVtech-capsule.tar.xz --overwrite --directory ./data
'''
if not os.path.isdir('./data/capsule'):
    for i in cmd.split('\n'):
        os.system(i)